In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25209
2,Huelva,Confirmados PDIA 14 días,471
3,Huelva,Tasa PDIA 14 días,"91,78245026014771"
4,Huelva,Confirmados PDIA 7 días,260
5,Huelva,Total Confirmados,25409
6,Huelva,Curados,20141
7,Huelva,Fallecidos,348


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25209.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6654.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3887 personas en los últimos 7 días 

Un positivo PCR cada 1918 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25209.0,260.0,471.0,513170.0,50.665471,91.782450,103.0
Condado-Campiña,7188.0,144.0,248.0,156231.0,92.171208,158.739303,60.0
Huelva-Costa,15293.0,103.0,201.0,289548.0,35.572686,69.418542,42.0
Almonte,955.0,61.0,80.0,24507.0,248.908475,326.437344,27.0
Huelva (capital),6654.0,37.0,75.0,143837.0,25.723562,52.142356,14.0
Cartaya,990.0,19.0,33.0,20083.0,94.607379,164.318080,9.0
Rociana del Condado,333.0,15.0,21.0,7939.0,188.940673,264.516942,9.0
Alosno,145.0,20.0,23.0,3933.0,508.517671,584.795322,9.0
Bollullos Par del Condado,642.0,11.0,24.0,14387.0,76.457913,166.817266,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Alosno,145.0,20.0,23.0,3933.0,508.517671,584.795322,9.0
San Bartolomé de la Torre,216.0,6.0,16.0,3761.0,159.532039,425.418772,4.0
Puebla de Guzmán,110.0,2.0,12.0,3092.0,64.683053,388.098318,2.0
Villalba del Alcor,459.0,4.0,13.0,3366.0,118.835413,386.215092,1.0
Almonte,955.0,61.0,80.0,24507.0,248.908475,326.437344,27.0
Rociana del Condado,333.0,15.0,21.0,7939.0,188.940673,264.516942,9.0
Bollullos Par del Condado,642.0,11.0,24.0,14387.0,76.457913,166.817266,6.0
Gibraleón,534.0,14.0,21.0,12737.0,109.915993,164.873989,3.0
Cartaya,990.0,19.0,33.0,20083.0,94.607379,164.318080,9.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puebla de Guzmán,110.0,2.0,12.0,3092.0,64.683053,388.098318,2.0,0.166667
Villalba del Alcor,459.0,4.0,13.0,3366.0,118.835413,386.215092,1.0,0.307692
Palos de la Frontera,465.0,4.0,13.0,11742.0,34.065747,110.713677,3.0,0.307692
Ayamonte,1237.0,6.0,17.0,21104.0,28.430629,80.553450,2.0,0.352941
San Bartolomé de la Torre,216.0,6.0,16.0,3761.0,159.532039,425.418772,4.0,0.375000
Lepe,2011.0,6.0,16.0,27880.0,21.520803,57.388809,2.0,0.375000
Bollullos Par del Condado,642.0,11.0,24.0,14387.0,76.457913,166.817266,6.0,0.458333
Isla Cristina,2006.0,8.0,17.0,21393.0,37.395410,79.465246,2.0,0.470588
Huelva (capital),6654.0,37.0,75.0,143837.0,25.723562,52.142356,14.0,0.493333
